# A few favorite recipes in computer vision & deep learning
> This blog post enlists a few of my favorite recipes in deep learning in the context of computer vision (as of August 2020).

- toc:true
- branch: master
- badges: false
- image: images/montage_vrl.png
- comments: true
- author: Sayak Paul
- categories: [visual-representation-learning, self-supervised-learning, computer-vision]

A few days ago from the time of writing this blog post I tweeted - 

> twitter: https://twitter.com/RisingSayak/status/1285866356592635904?s=20)

In this blog post, I will expand on this tweet to convey why these are my favorite recipes among other things. 

The training frameworks I mentioned can be classified into two broad categories -

- **supervised learning** (Supervised Contrastive Learning [1] and BigTransfer [2])
- **self-supervised learning** (SimCLRv2 [only the *SimCLR* part]).

> Note: that SimCLR [3] and SimCLRv2 [4] are two separate works. 

# So, why self-supervised learning anyway?

The field of self-supervised visual representation learning is progressing pretty fast. With recent advancements, the deep learning community has started to consider it as an alternative to the fully supervised models for tasks like image classification, object detection, image segmentation, etc. If you are unfamiliar with self-supervised learning check out [this blog post](https://www.fast.ai/2020/01/13/self_supervised/) by Jeremy Howard. So, ***why all the fuss around self-supervised visual representation learning?***

This is because a self-supervised learning framework can benefit from ***unlabeled data***. Essentially, you would frame a supervised learning task from a large unlabeled corpus and then train a model to learn that task. You see we are ***not*** using any explicit label information here. Instead, we are using the given data to form a supervised learning task, this is why it is self-supervised. You would then take the representations from the model (preferably from the encoder part of the model) and use them for downstream tasks. Representations learned using self-supervised learning frameworks like SimCLRv2, SwAV [5] transfer quite well to downstream (vision) tasks even with very less labeled data.

[Colin Raffel](https://colinraffel.com/) beautifully summed up the recent progress in the field of self-supervised learning for computer vision - 

> twitter: https://twitter.com/colinraffel/status/1289315020199743488?s=20

BYOL [6] and SwAV have even beaten SimCLR - 

![](https://github.com/sayakpaul/portfolio/raw/master/images/byol_swav.png "We see that training frameworks like BYOL and SwAV have already beaten SimCLR and all of this (including the SimCLR) has happened in 2020 itself! (Sources: BYOL and SwAV papers respectively [6][5])")

One could argue that this is in comparison with SimCLR but not SimCLRv2. Well, that is because SimCLRv2 is not just about self-supervised learning, it is more than that - 

![](https://github.com/sayakpaul/portfolio/raw/master/images/simclr_v2.png "Source: SimCLRv2 [4]")


This blog post, however, would only focus on the SimCLR part i.e. the left-most part of the figure above. If you are interested to know more about SimCLRv2, feel free to watch [this video](https://www.youtube.com/watch?v=2lkUNDZld-4&lc) by Yannic Kilcher. 

# Why SimCLR(v2)?

Among all of the techniques that have been mentioned in the post so far, SimCLR is by far the most favorite of mine.  This is because of its simplicity and the promise to shine more as one would scale up the base architecture and introduce more (unlabeled data). Moreover, framework wise SimCLR is very consistent with the recipes I would want to see in an effective visual representation learning framework. One of these primary recipes is to maximize the agreement between the semantically similar images. SwAV is also capable of doing this, in fact, it is currently the state-of-the-art (as of August 2020) in this domain. But simplicity wise SimCLR beats SwAV big time.

At a very high-level, SimCLR takes two different views of the same image and tries to maximize the agreement between these two views while minimizing the agreement between the views coming from other images. These different views are obtained by applying augmentation operations like random-resized crops, horizontal flips, color distortions, etc. Representations learned using this framework (and any self-supervised visual representation learning framework in general) can be used in different flavors - 

- You may have loads of unlabeled data and limited labeled data for your problem. You could use the unlabeled data and incorporate SimCLR for obtaining effective representations and use them to a downstream task where the limited labeled data might be required.
- Representations obtained from datasets like ImageNet using SimCLR can be used in regular transfer learning settings.

![](https://camo.githubusercontent.com/d92c0e914af70fe618cf3ea555e2da1737d84bc4/68747470733a2f2f312e62702e626c6f6773706f742e636f6d2f2d2d764834504b704539596f2f586f3461324259657276492f414141414141414146704d2f766146447750584f79416f6b4143385868383532447a4f67457332324e68625877434c63424741735948512f73313630302f696d616765342e676966 "Source: [SimCLR blog post](https://ai.googleblog.com/2020/04/advancing-self-supervised-and-semi.html)")

As we can see in SimCLR, the loss function (normalized temperature-scaled cross-entropy loss) operates directly on the features computed by the projection head (MLP part). This makes SimCLR a compute-intensive framework. 

On the other hand, SwAV operates by assigning the encoded representations of different views of the same image to clusters. The clusters are being assigned by keeping a ***differentiable*** codebook for the prototypes of the different types of images present in the given dataset. Training wise, SwAV tries to maximize the agreement between the clusters of semantically similar images. Operating on the clusters rather than the encoded representations is a lesser compute-intensive task. 

![](https://camo.githubusercontent.com/a6d39dcc04416c84bd23f5e2ae87dceb40c6ef8b/68747470733a2f2f646c2e666261697075626c696366696c65732e636f6d2f64656570636c75737465722f616e696d617465642e676966 "Source: [GitHub repository](https://github.com/facebookresearch/swav/) of SwAV")

SwAV might appear as a simpler framework than SimCLR but there are a number of different pieces to look after here:

- Maintaining a differentiable codebook
- Optimal transportation of the representations to form the codebook using the [Sinkhorn-Knopp algorithm](https://en.wikipedia.org/wiki/Sinkhorn%27s_theorem)
- Multi crop data augmentation policy
- Swapping prediction problem

When working in practical scenarios we often need to maintain a trade-off between technical feasibility and performance. To me, SimCLR cuts it through in terms of technical feasibility. 

# Returning to supervised learning

Given the almightly prowess of the self-supervised learning frameworks why even bother about supervised regimes? 

## Labels + contrastive loss = win-win

Supervised Contrastive Learning addresses a very important point about the self-supervised learning frameworks like SimCLR. In SimCLR, the positive pairs are generated by taking different views of the same image and the negative pairs are then randomly sampled from the other images present in a batch. 


![](https://github.com/sayakpaul/portfolio/raw/master/images/positive_pairs.png "Examples of positive pairs (Source: [Exploring SimCLR](https://sthalles.github.io/simple-self-supervised-learning/))")

Here are some examples of negative pairs - 


![](https://github.com/sayakpaul/portfolio/raw/master/images/negative_pairs.png "Source: [Exploring SimCLR](https://sthalles.github.io/simple-self-supervised-learning/)")


As discussed in the Supervised Contrastive Learning paper, this method of contrasting two different views of the same image can result in false negatives i.e. the samples belonging to the same class might get mapped differently in the embedding space. There's no way for us to properly mitigate this issue without having access to the original labels. Hence, I mentioned if you have loads of labeled images, it's better to use Supervised Contrastive Learning to capture meaningful representations. 

![](https://github.com/sayakpaul/portfolio/raw/master/images/scl_embeddings.png "Source: Supervised Contrastive Learning [1]")


Supervised Contrastive Learning extends how we train supervised models by introducing a two stage training framework. In the first stage, it uses the label information in the contrastive loss to learn to map the encoded representations effectively. In the second stage, it train a linear model  on top of these encoded representations for the given supervised training objective. 

In practice this works quite well - 

![](https://github.com/sayakpaul/portfolio/raw/master/images/scl_results.png "Source: Supervised Contrastive Learning [1]")


You might be feeling very tempted to try out this on your labeled dataset. Truth be told - Supervised Contrastive Learning is also compute-intensive even for relatively small datasets. So, if you don't have the training budget you might need to reconsider this option. 

Maybe you have a relatively smaller labeled dataset and you just want to be able to fine-tune a good enough architecture which is still very useful for many practical scenarios. This is where BigTransfer can really shine. 

# Returning to supervised transfer learning

BigTransfer is from the family of classic supervised pre-training and transfer the learned representations to downstream tasks. ImageNet has been the choice for this kind of pre-training for a long time. But in BigTransfer, the authors use larger datasets such as ImageNet-21k, JFT along with ImageNet. In order to facilitate these larger datasets they scale up the model architectures along with longer pre-training. Their result speaks for itself - 

![](https://github.com/sayakpaul/portfolio/raw/master/images/bit_results.png "Source: [BigTransfer blog post](https://blog.tensorflow.org/2020/05/bigtransfer-bit-state-of-art-transfer-learning-computer-vision.html)")

To eliminate the dependence on batch statistics the authors make use of Group Normalization and Weight Standardization.  Personally, I really liked this recipe because using overall large batch sizes to train larger models at scale is a common choice and using Batch Normalization there could have easily affected the performance of the models during the downstream tasks. 

For fine-tuning, the authors propose a heuristics-based BiT-HyperRule which provides instructions on what augmentation policy to use, how many steps to train for, what learning rate schedule to use, etc.  

![](https://github.com/sayakpaul/portfolio/raw/master/images/bit_hyperrule.png "Source: [BigTransfer blog post](https://blog.tensorflow.org/2020/05/bigtransfer-bit-state-of-art-transfer-learning-computer-vision.html)")


I found this strategy to be simple enough to be implemented and practised. 

# Resources

If you are interested to apply these techniques in your own works you may find the following resources to be helpful - 

- [A Colab Notebook](https://colab.research.google.com/github/google-research/simclr/blob/master/colabs/finetuning.ipynb) by the authors of SimCLRv2 that shows how to fine-tune with SimCLRv2.
- [A report](https://bit.ly/2UVZtm7) by Sweta Shaw and myself that walks through Supervised Contrastive Learning along with Colab Notebooks.
- [A tutorial](https://blog.tensorflow.org/2020/05/bigtransfer-bit-state-of-art-transfer-learning-computer-vision.html) on BigTransfer by the authors of BigTransfer.

# Acknowledgments

Amit's [visual guide on SimCLR](https://amitness.com/2020/03/illustrated-simclr/) and [Thalles's exploratory analysis on SimCLR](https://sthalles.github.io/simple-self-supervised-learning/) helped me in developing a thorough understanding of SimCLR. 

Thanks to Yannic Kilcher for his explanation videos on [SimCLRv2](https://www.youtube.com/watch?v=2lkUNDZld-4&lc) and [Supervised Contrastive Learning](https://www.youtube.com/watch?v=MpdbFLXOOIw). Those made the learning process smoother. 

Thanks to Ting Chen (SimCLR author) for providing me with additional pointers on self-supervised learning in general.  

Thanks to Jeremy Howard for his [fast.ai](http://fast.ai) lectures that continue to help me in approaching deep learning recipes with more common sense and practicality. 

# References

1. Khosla, Prannay, et al. “Supervised Contrastive Learning.” ArXiv:2004.11362 [Cs, Stat], Apr. 2020. [arXiv.org](http://arxiv.org/), [http://arxiv.org/abs/2004.11362](http://arxiv.org/abs/2004.11362).
2. Kolesnikov, Alexander, et al. “Big Transfer (BiT): General Visual Representation Learning.” ArXiv:1912.11370 [Cs], May 2020. [arXiv.org](http://arxiv.org/), [http://arxiv.org/abs/1912.11370](http://arxiv.org/abs/1912.11370).
3. Chen, Ting, Simon Kornblith, Mohammad Norouzi, et al. “A Simple Framework for Contrastive Learning of Visual Representations.” ArXiv:2002.05709 [Cs, Stat], June 2020. [arXiv.org](http://arxiv.org/), [http://arxiv.org/abs/2002.05709](http://arxiv.org/abs/2002.05709).
4. Chen, Ting, Simon Kornblith, Kevin Swersky, et al. “Big Self-Supervised Models Are Strong Semi-Supervised Learners.” ArXiv:2006.10029 [Cs, Stat], June 2020. [arXiv.org](http://arxiv.org/), [http://arxiv.org/abs/2006.10029](http://arxiv.org/abs/2006.10029).
5. Caron, Mathilde, et al. “Unsupervised Learning of Visual Features by Contrasting Cluster Assignments.” ArXiv:2006.09882 [Cs], July 2020. [arXiv.org](http://arxiv.org/), [http://arxiv.org/abs/2006.09882](http://arxiv.org/abs/2006.09882).
6. Grill, Jean-Bastien, et al. “Bootstrap Your Own Latent: A New Approach to Self-Supervised Learning.” ArXiv:2006.07733 [Cs, Stat], June 2020. [arXiv.org](http://arxiv.org/), [http://arxiv.org/abs/2006.07733](http://arxiv.org/abs/2006.07733).